This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [3]:
!pip install python-dotenv

In [117]:
# get api key from your .env file
import os
from dotenv import load_dotenv  # if missing this module, simply run `pip install python-dotenv`

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')


Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [76]:
# First, import the relevant modules
import requests
import math

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [29]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
#URL = "https://data.nasdaq.com/api/v3/datasets/XFRA/AFX/EOD.json?start_date=2023-01-01&end_date=2023-01-01"
URL =  f"https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json?api_key={API_KEY}"
PARAMS = {
    "start_date": "2007-01-03",
    "end_date": "2007-01-03",
}
response = requests.get(url=URL, params=PARAMS)
response.raise_for_status
data_sample = response.json()
print(data_sample)

{'dataset_data': {'limit': None, 'transform': None, 'column_index': None, 'column_names': ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover'], 'start_date': '2007-01-03', 'end_date': '2007-01-03', 'frequency': 'daily', 'data': [['2007-01-03', 16.5, 16.65, 16.28, 16.33, None, 57386.0, 944194.0, None, None, None]], 'collapse': None, 'order': None}}


In [30]:
print(data_sample["dataset_data"]["data"])

[['2007-01-03', 16.5, 16.65, 16.28, 16.33, None, 57386.0, 944194.0, None, None, None]]


In [32]:
print(data_sample["dataset_data"]["column_names"])

['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover']


In [9]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure

{'dataset': {'id': 10095370, 'dataset_code': 'AFX_X', 'database_code': 'FSE', 'name': 'Carl Zeiss Meditec (AFX_X)', 'description': 'Stock Prices for Carl Zeiss Meditec (2020-11-02) from the Frankfurt Stock Exchange.<br><br>Trading System: Xetra<br><br>ISIN: DE0005313704', 'refreshed_at': '2020-12-01T14:48:09.907Z', 'newest_available_date': '2020-12-01', 'oldest_available_date': '2000-06-07', 'column_names': ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover'], 'frequency': 'daily', 'type': 'Time Series', 'premium': False, 'limit': None, 'transform': None, 'column_index': None, 'start_date': '2021-01-03', 'end_date': '2020-12-01', 'data': [], 'collapse': None, 'order': None, 'database_id': 6129}}


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [106]:
# 1. Collect Data for AFX_X for 2017

URL =  f"https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json?api_key={API_KEY}"
PARAMS = {
    "start_date": "2017-01-01",
    "end_date": "2018-01-01",
    "order": "asc"
}
response = requests.get(url=URL, params=PARAMS)
response.raise_for_status
data = response.json()


In [107]:
# 2. Convert Returned JSON object into python dictionary
data_list = data["dataset_data"]["data"]

for day in data_list:
    data_dict[day[0]] =  {'Open': day[1], 
               'High': day[2],
               'Low': day[3],
               'Close': day[4],
               'Change': day[5],
               'Traded Volume': day[6],
               'Turnover': day[7],
               'Last Price of the Day': day[8], 
               'Daily Traded Units': day[9],
               'Daily Turnover': day[10]
        } 


In [108]:
print(data_dict["2017-01-02"])

{'Open': 34.99, 'High': 35.94, 'Low': 34.99, 'Close': 35.8, 'Change': None, 'Traded Volume': 44700.0, 'Turnover': 1590561.0, 'Last Price of the Day': None, 'Daily Traded Units': None, 'Daily Turnover': None}


In [50]:
# 3. Calculate highest and lowest prices in this period
high = 0
low = 10000

for day in data_dict.values():
    if day["High"]> high:
        high = day["High"]
    if day["Low"]< low:
        low = day["Low"]
        
print(f"High: {high} | Low: {low}")
    

High: 53.54 | Low: 33.62


In [52]:
# 4. Calculate largest change in 1 day
change_max = 0

for day in data_dict.values():
    if day["High"]-day["Low"]> change_max:
        change_max = day["High"]-day["Low"]
    
print(f"Max Change: {round(change_max,2)}")


Max Change: 2.81


In [112]:
# 5. Largest Change Between 2 Days
change2 = 0
prev_day = data_dict["2017-01-02"]["Close"]

for day in data_dict.values():
    if abs(day["Close"] - prev_day) >abs(change2):
        change2 = day["Close"] - prev_day   
    prev_day = day["Close"]
    
print(f"Max Change: ${round(change2,2)}")

Max Change: $-2.56


In [113]:
# 6. What was the average daily trading volume for this year?
vol_sum = 0
num_days = 0
for day in data_dict.values():
    vol_sum += day["Traded Volume"]
    num_days += 1
    
ADT_vol = vol_sum/num_days
print(f"Average Daily Traded Volume: {round(ADT_vol,2)}")
    

Average Daily Traded Volume: 89124.34


In [114]:
# 7. What was the median trading volume for the whole year?
vol_list = []
#Turn volumes into list
for day in data_dict.values():
    vol_list.append(day["Traded Volume"])
    

    
    

In [115]:
# 7. What was the median trading volume for the whole year?
def find_median(list):
    list.sort()
    midpoint_ind = len(list)/2
    if len(list)%2 == 0: #Handles even number of entries
        median = (list[midpoint_ind]+list[midpoint_ind-1])/2
    else: #Handles odd number of entries
        median = list[math.floor(midpoint_ind)]
    return median
            

In [116]:
# 7. What was the median trading volume for the whole year?
vol_median = find_median(vol_list)
print(vol_median)

76286.0
